In [9]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import datetime
from hello import hello_logger
import os
import requests
import time

In [10]:
# General functions

def call(f, *a, _quiet=False, **k):
    if not _quiet:
        now = datetime.datetime.now().strftime("%H:%M:%S")
        s = "(" + ", ".join(map(repr, a)) + ")"
        print("%s Calling %s%s" % (now, f.__name__, s))
    while True:
        try:
            rv = f(*a, **k)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.Timeout as e:
            print("Got timeout! Make sure you're on the right network!")
        except (BadError, IOError, requests.exceptions.ConnectionError) as e:
            print("Error occurred:", e.__class__, e.args[0])
        else:
            return rv        
            
def connect(ip, *args):
    global h
    h = HelloApp(ip, *args)
    
def dfmt(d):
    return d.strftime("%m/%d/%Y %I:%M:%S %p")
    
def wait1(s):
    if s <= 0: return
    dend = dfmt(datetime.datetime.now() + datetime.timedelta(seconds=s))
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    while time.time() < end:
        time.sleep(5)
    
def wait3(s, src):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=s)
    m1 = dfmt(start)
    m2 = dfmt(end)
    print("\r(%s) Sleeping %d seconds from %s to %s          " % (src, s, m1, m2), end="")
    tend = time.time() + s
    while time.time() < tend:
        time.sleep(5)
    
def wait2(s):
    pass
wait = wait1

In [18]:
# Specific functions

# globals
do_o2_req  = 0
do_n2_req  = 0
ph_co2_req = 0
batch_tmpl = "domcr t %d"
logger = None

def setconfig(g, s, v):
    call(h.setconfig, g, s, v)

def getmv():
    return call(h.gpmv, _quiet=True)
    
def setup_gases():
    global do_o2_req, do_n2_req, ph_co2_req
    setconfig("pH", 'CO2_Manual_Max_(%)', 100)
    setconfig("DO", 'O2_Manual_Max_(%)', 100)
    setconfig("DO", 'N2_Manual_Max_(%)', 100)
    mv = getmv()
    do_o2_req = mv['do']['outputUp']
    do_n2_req = mv['do']['outputDown']
    ph_co2_req = mv['ph']['outputDown']
    
def setup_controls(rpm=18, temp=37):
    setag(rpm)
    settemp(temp)   
    
def maingas(v):
    qset(h.setmg, v)
    
def qset(f, v):
    if v > 0:
        call(f, 1, v)
    else:
        call(f, 2, 0)
        
def main_n2(pc):
    setdo(pc, do_o2_req)
    
def micro_n2(pc):
    global ph_co2_req
    if pc > 0:
        call(h.setph, 1, pc, 0)
    else:
        call(h.setph, 2, ph_co2_req, 0)
    ph_co2_req = pc
    
def phoff():
    call(h.setph, 2, 0, 0)
    
def micro_o2(pc):
    setdo(do_n2_req, pc)
    
def setdo(n2, o2):
    global do_n2_req, do_o2_req
    call(h.setdo, 1, n2, o2)
    do_o2_req = o2
    do_n2_req = n2
    
def setag(rpm):
    call(h.setag, 0, rpm)
    
def settemp(t):
    call(h.settemp, 0, t)
    
def endbatch():
    if call(h.batchrunning):
        call(h.endbatch)
    
def startbatch(n):
    endbatch()
    call(h.startbatch, n)
    
def docal1pt():
    v = call(h.getrawvalue, "doa")
    call(h.trycal, 'doa', v, 100)
    call(h.savetrialcal, 'doa')

def dopv():
    return getmv()['do']['pv']

def waitdo_lower(t, timeout=8*3600):
    while dopv() > t and timeout>0:
        wait3(60, 'waitdo_lower')
        timeout -= 60
    print()
    
def dooff():
    call(h.setdo, 2, 0, 0)
    
def waitdo_higher(t, timeout=8*3600):
    while dopv() < t and timeout>0:
        wait3(60, 'waitdo_higher')
        timeout -= 60
    print()
    
def rampdown(mg=0.5, min=20, timeout=3*3600):
    print("rampdown(%r, %r, %r)"%(mg, min, timeout))
    maingas(mg)
    micro_n2(100)
    dooff()
    waitdo_lower(min, timeout)
    phoff()
    
def flushheadspace(gas='air', mg=0.5, wt=5*60):
    print("flushheadspace(%r, %r, %r)"%(gas, mg, wt))
    maingas(mg)
    if gas == "n2":
        main_n2(100)
    else:
        if do_n2_req != 0 or do_o2_req != 0:
            dooff()
    wait(wt)
    dooff()
    
def rampup(mg=0.5, o2=20, max=80, timeout=3*3600):
    print("rampup(%r, %r, %r, %r)"%(mg, o2, max, timeout))
    maingas(mg)
    micro_o2(o2)
    micro_n2(0)
    main_n2(0)
    waitdo_higher(max, timeout)
    dooff()
    
def newbatch(i, j):
    startbatch(batch_tmpl%(i,j))

In [19]:
def setup_logger(interval=10):
    global logger
    keys = [
        'temperature.pv',
        'agitation.pv',
        'ph.outputDown',
        'ph.manDown',
        'do.manUp',
        'do.outputDown',
        'do.pv',
        'do.manDown',
        'do.outputUp',
        'maingas.man',
        'maingas.mode',
        'maingas.pv',
        'MFCs.air',
        'MFCs.n2',
        'MFCs.o2',
        'MFCs.co2',
        'level.pv'
    ]
    if logger is not None:
        logger.stop()
    logger = hello_logger.HelloLogger(h, interval, None, keys)
    logger.start()
    
def logfile(fn):
    print("Creating new file: %r"%fn)
    logger.new_file(fn)
    
def endlog():
    logger.new_file(None)

In [20]:
def calibrate(t):
    maingas(0.5)
    micro_o2(100)
    micro_n2(0)
    main_n2(0)
    if t < 5400:
        wait(t/2)
        micro_o2(0)
        wait(t/2)
    else:
        wait(t-3600)
        micro_o2(0)
        wait(3600)
    docal1pt()
    
def ntests(n):
    return range(1,n+1)

### Test Utilities

In [ ]:
fn_tmpl = "DO micro test %s %s %s.csv"

def new_file(typ, name):
    logfile(fn_tmpl% (typ, name, bt()))

def dotest(mg, o2, i):
    new_file("rampdown", i)
    rampdown(0.5)
    endlog()
    flushheadspace('air')
    new_file("rampup", i)
    rampup(mg, o2)
    endlog()
    dooff()
    
def bt(): 
    return datetime.datetime.now().strftime("%y%m%d")

def mkfldr(p):
    path = "C:\\Users\\natha\\Documents\\test\\"+p
    os.makedirs(path, exist_ok=True)
    os.chdir(path)
    
def chd(p):
    os.chdir("C:\\Users\\natha\\Documents\\test\\"+p)

### Pre-Test Setup

In [ ]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
mkfldr("Sam Microsparger Test")

In [ ]:
calibrate(1*3600)
dotest(0.1, 100, "dip tube 2")    

In [ ]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
chd("Sam Microsparger Test")
for i in range(1,4):
    dotest(0.1, 100, "L plate A %d"%i)    

In [ ]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
calibrate(0)
chd("Sam Microsparger Test")
for i in range(1,4):
    dotest(0.125, 100, "L plate A 0.125LPM %d"%i)    

In [ ]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
calibrate(300)
chd("Sam Microsparger Test")
for i in range(1,4):
    dotest(0.2, 100, "L plate A 0.2LPM %d"%i)    

In [ ]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
mkfldr("Sam Microsparger Test\\0.1-0.2 overnight 170815")
calibrate(3600*3)
for i in ntests(3):
    for mg in (0.1, 0.125, 0.15, 0.175, 0.2):
        dotest(mg, 100, "L plate A %.3fLPM %d"%(mg,i))    

In [ ]:
connect('192.168.1.81')
setup_gases()
setup_controls(32, 37)
setup_logger()
mkfldr("Sam Microsparger Test\\L Plate C 0.1-0.2 overnight 170817")
calibrate(3600*3)
for i in ntests(3):
    for mg in (0.1, 0.125, 0.15, 0.175, 0.2):
        dotest(mg, 100, "L plate A %.3fLPM %d"%(mg,i))    

In [ ]:
connect('192.168.1.17')
setup_gases()
setup_controls(32, 37)
setup_logger()
mkfldr("Sam Microsparger Test\\L Plate D 0.1-0.2 overnight 170831")
calibrate(3600*6)
for i in ntests(3):
    for mg in (0.1, 0.125, 0.15, 0.175, 0.2):
        dotest(mg, 100, "L plate D %.3fLPM %d"%(mg,i))    

In [18]:
connect('192.168.1.17')
setup_gases()
setup_controls(32, 37)
setup_logger()
plate = "D 2mm"
mkfldr("Sam Microsparger Test\\L Plate %s 0.1-0.2 overnight 170831"%plate)
calibrate(3600*6)
for i in ntests(3):
    for mg in (0.1, 0.125, 0.15, 0.175, 0.2):
        dotest(mg, 100, "L plate %s %.3fLPM %d"%(plate, mg,i))    

13:57:09 Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100)
13:57:10 Calling setconfig('DO', 'O2_Manual_Max_(%)', 100)
13:57:10 Calling setconfig('DO', 'N2_Manual_Max_(%)', 100)
13:57:11 Calling setag(0, 32)
13:57:12 Calling settemp(0, 37)
13:57:13 Calling setmg(1, 0.5)
13:57:15 Calling setdo(1, 0, 100)
13:57:16 Calling setph(2, 0, 0)
13:57:16 Calling setdo(1, 0, 100)
Sleeping 18000 seconds, ending at 09/05/2017 06:57:17 PM
18:57:18 Calling setdo(1, 0, 0)
Error occurred: <class 'hello.hello3.ServerCallError'>
Sleeping 3600 seconds, ending at 09/05/2017 07:57:20 PM
19:57:20 Calling getRawValue('doa')
Error occurred: <class 'hello.hello3.ServerCallError'>
19:57:20 Calling trycal('doa', 97.700912, 100)
19:57:20 Calling savetrialcal('doa')
Creating new file: 'DO micro test rampdown L plate D 2mm 0.100LPM 1 170905.csv'
rampdown(0.1, 20, 7200)
19:57:21 Calling setmg(1, 0.1)
19:57:21 Calling setph(1, 100, 0)
19:57:22 Calling setdo(2, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 09/05/2017 08

In [20]:
connect('192.168.1.17')
setup_gases()
setup_controls(32, 37)
setup_logger()
plate = "D 3mm"
mkfldr("Sam Microsparger Test\\L Plate %s 0.1-0.2 overnight %s"%(plate, bt()))
calibrate(3600*8)
for i in ntests(3):
    for mg in (0.1, 0.125, 0.15, 0.175, 0.2):
        dotest(mg, 100, "L plate %s %.3fLPM %d"%(plate, mg,i))    

11:59:16 Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100)
11:59:17 Calling setconfig('DO', 'O2_Manual_Max_(%)', 100)
11:59:17 Calling setconfig('DO', 'N2_Manual_Max_(%)', 100)
11:59:18 Calling setag(0, 32)
11:59:19 Calling settemp(0, 37)
11:59:20 Calling setmg(1, 0.5)
11:59:22 Calling setdo(1, 0, 100)
11:59:23 Calling setph(2, 0, 0)
11:59:23 Calling setdo(1, 0, 100)
Sleeping 25200 seconds, ending at 09/07/2017 06:59:24 PM
18:59:26 Calling setdo(1, 0, 0)
Error occurred: <class 'hello.hello3.ServerCallError'>
Sleeping 3600 seconds, ending at 09/07/2017 07:59:27 PM
19:59:28 Calling getRawValue('doa')
Error occurred: <class 'hello.hello3.ServerCallError'>
19:59:28 Calling trycal('doa', 99.786743, 100)
19:59:29 Calling savetrialcal('doa')
Creating new file: 'DO micro test rampdown L plate D 3mm 0.100LPM 1 170907.csv'
rampdown(0.1, 20, 7200)
19:59:29 Calling setmg(1, 0.1)
19:59:29 Calling setph(1, 100, 0)
19:59:30 Calling setdo(2, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 09/07/2017 08

07:09:06 Calling setdo(1, 0, 100)
07:09:07 Calling setph(2, 100, 0)
07:09:07 Calling setdo(1, 0, 100)
(waitdo_higher) Sleeping 60 seconds from 09/08/2017 08:05:16 AM to 09/08/2017 08:06:16 AM          
08:06:16 Calling setdo(2, 0, 0)
Error occurred: <class 'hello.hello3.ServerCallError'>
08:06:17 Calling setdo(2, 0, 0)
Creating new file: 'DO micro test rampdown L plate D 3mm 0.175LPM 2 170908.csv'
rampdown(0.175, 20, 7200)
08:06:17 Calling setmg(1, 0.175)
08:06:18 Calling setph(1, 100, 0)
08:06:19 Calling setdo(2, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 09/08/2017 08:33:23 AM to 09/08/2017 08:34:23 AM          
08:34:23 Calling setph(2, 0, 0)
flushheadspace('air', 0.5, 300)
08:34:24 Calling setmg(1, 0.5)
08:34:25 Calling setdo(2, 0, 0)
Sleeping 300 seconds, ending at 09/08/2017 08:39:25 AM
08:39:25 Calling setdo(2, 0, 0)
Creating new file: 'DO micro test rampup L plate D 3mm 0.175LPM 2 170908.csv'
rampup(0.175, 100, 80, 7200)
08:39:25 Calling setmg(1, 0.175)
08:39:26 Calling setd

In [24]:
connect('192.168.1.17')
setup_gases()
setup_controls(32, 37)
setup_logger()
plate = "E 4mm"
mkfldr("Sam Microsparger Test\\L Plate %s 0.1-0.2 overnight %s"%(plate, bt()))
calibrate(8*3600)
for i in ntests(3):
    for mg in (0.1, 0.125, 0.15, 0.175, 0.2):
        dotest(mg, 100, "L plate %s %.3fLPM %d"%(plate, mg,i))    

16:56:59 Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100)
16:57:00 Calling setconfig('DO', 'O2_Manual_Max_(%)', 100)
16:57:01 Calling setconfig('DO', 'N2_Manual_Max_(%)', 100)
16:57:01 Calling setag(0, 32)
16:57:02 Calling settemp(0, 37)
16:57:03 Calling setmg(1, 0.5)
16:57:05 Calling setdo(1, 0, 100)
16:57:06 Calling setph(2, 0, 0)
16:57:06 Calling setdo(1, 0, 100)
Sleeping 25200 seconds, ending at 09/15/2017 11:57:07 PM
23:57:09 Calling setdo(1, 0, 0)
Error occurred: <class 'hello.hello3.ServerCallError'> Cannot perform call set - user  is missing the following permissions: Hello Access, Controls, Remote Access
Custom Error Message.vi->Determine User Permission for Call.vi->Webservice Command Handler.vi->PBS_WebServer.vi->PBS_WebServer.vi.ProxyCaller 5033
Sleeping 3600 seconds, ending at 09/16/2017 12:57:10 AM
00:57:11 Calling getRawValue('doa')
Error occurred: <class 'hello.hello3.ServerCallError'> Cannot perform call getrawvalue - user  is missing the following permissions: Hello

Creating new file: 'DO micro test rampup L plate E 4mm 0.200LPM 1 170916.csv'
rampup(0.2, 100, 80, 10800)
09:58:03 Calling setmg(1, 0.2)
09:58:04 Calling setdo(1, 0, 100)
09:58:05 Calling setph(2, 100, 0)
09:58:05 Calling setdo(1, 0, 100)
(waitdo_higher) Sleeping 60 seconds from 09/16/2017 11:00:14 AM to 09/16/2017 11:01:14 AM          
11:01:14 Calling setdo(2, 0, 0)
Error occurred: <class 'hello.hello3.ServerCallError'> Cannot perform call set - user  is missing the following permissions: Hello Access, Controls, Remote Access
Custom Error Message.vi->Determine User Permission for Call.vi->Webservice Command Handler.vi->PBS_WebServer.vi->PBS_WebServer.vi.ProxyCaller 5033
11:01:15 Calling setdo(2, 0, 0)
Creating new file: 'DO micro test rampdown L plate E 4mm 0.100LPM 2 170916.csv'
rampdown(0.5, 20, 10800)
11:01:15 Calling setmg(1, 0.5)
11:01:16 Calling setph(1, 100, 0)
11:01:17 Calling setdo(2, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 09/16/2017 11:37:24 AM to 09/16/2017 11:38:24

21:00:19 Calling setdo(2, 0, 0)
Creating new file: 'DO micro test rampdown L plate E 4mm 0.100LPM 3 170916.csv'
rampdown(0.5, 20, 10800)
21:00:19 Calling setmg(1, 0.5)
21:00:20 Calling setph(1, 100, 0)
21:00:21 Calling setdo(2, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 09/16/2017 09:35:26 PM to 09/16/2017 09:36:26 PM          
21:36:26 Calling setph(2, 0, 0)
Error occurred: <class 'hello.hello3.ServerCallError'> Cannot perform call set - user  is missing the following permissions: Hello Access, Controls, Remote Access
Custom Error Message.vi->Determine User Permission for Call.vi->Webservice Command Handler.vi->PBS_WebServer.vi->PBS_WebServer.vi.ProxyCaller 5033
flushheadspace('air', 0.5, 300)
21:36:27 Calling setmg(1, 0.5)
21:36:28 Calling setdo(2, 0, 0)
Sleeping 300 seconds, ending at 09/16/2017 09:41:28 PM
21:41:28 Calling setdo(2, 0, 0)
Creating new file: 'DO micro test rampup L plate E 4mm 0.100LPM 3 170916.csv'
rampup(0.1, 100, 80, 10800)
21:41:29 Calling setmg(1, 0.1)
21:41: